In [2]:
import ee 
from datetime import datetime
from datetime import timedelta
import os
import geemap
import logging
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import ee_helpers

In [ ]:
# Trigger the authentication flow
ee.Authenticate()

# Initialize the library
ee.Initialize()

In [3]:
glacier_view_dir = os.path.join(os.path.expanduser('~'),"Desktop","projects","GlacierView")
glims_bb_path = os.path.join(glacier_view_dir,"src","glims","data","training_sample","glims_18k_bb.shp")
df = gpd.read_file(glims_bb_path)

In [ ]:
#only use if you want to resume training

#done_df = pd.read_csv("/Users/mattw/Desktop/projects/gee-pull/data/training_logs/Batch 1/training_log.log", header = None)
#done_glims_ids = [row.split(":")[3][1:] for row in done_df[1]]

#df = df[~df.glac_id.isin(done_glims_ids)]

In [4]:
df = df.sample(frac=1).reset_index(drop=True)
n = df.shape[0]

In [5]:
n

18093

In [ ]:
for idx,row in enumerate(df.iterrows()):
    glims_id = row[1].glac_id
    bounding_box = eval(row[1].bboxes)
    date = row[1].src_date.split("T")[0]
    date = datetime.strptime(date, '%Y-%m-%d')
    out_dir = os.path.join(glacier_view_dir,"src","earth_engine","data","ee_landing_zone",
                           "localized_time_series_for_segmentation_training_large","landsat",glims_id)
    if not os.path.exists(out_dir):
        os.mkdir(out_dir)
    
    start_date = date - timedelta(days = 50)
    end_date = date + timedelta(days = 50)
    try:
        ee_helpers.export_landsat_five_images(glims_id,
                                                  bounding_box,
                                                  start_date,
                                                  end_date,
                                                  out_dir,
                                                  cloudy_pixel_percentage = 200,
                                             )
    except:
        pass
    
    try:
        ee_helpers.export_landsat_seven_images(glims_id,
                                              bounding_box,
                                              start_date,
                                              end_date,
                                              out_dir,
                                              cloudy_pixel_percentage = 200,
                                              )
    except:
        pass
        
    try:
        ee_helpers.export_landsat_eight_images(glims_id,
                                              bounding_box,
                                              start_date,
                                              end_date,
                                              out_dir,
                                              cloudy_pixel_percentage = 200,
                                              )
    except:
        pass
    
    print(f"################ COMPLETED PULL FOR {glims_id}, {idx+1}/{n} REMAINING ################")

In [ ]:
os.mkdir("data/training/dems")
for row in df.iterrows():
    glims_id = row[1].glac_id
    bounding_box = eval(row[1].bboxes)
    out_dir = os.path.join("data","training","dems")
    try:
        ee_helpers.export_nasa_dems(glims_id, bounding_box, out_dir)
    except:
        pass

In [ ]:
pd.Series(completed).to_csv("completed_batch_1.csv")